In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
from datetime import datetime
from datetime import timedelta

<h1> A Machine Learning Model for Solar Flare Predictions </h1>

<h2>I. Overview</h2>

<h3> What is a solar flare? </h3>

A solar flare is a powerful electromagnetic radioactive eruption of the Sun. It occurs in active regions(AR) which are charactirized with magnetic abnormlities - extremely strong and complex magnetic field. A solar flare can release energy of $10^{21}$ to $10^{25}$ J. This leads to disruptions in radio communication technologies and even fallouts. For example, the biggest solar flare ever recorded, happened in 1859 and caused auroras, which could be seen from the Carribean, blackouts and failiure of most telegraph systems over North America and Europe. That is why scientist are trying to find a way to predict when an event like this would happen.

In [3]:
#date format 2022.11.01_00:00:00-2022.11.02_00:00:00
import drms

def extractHMI_SHARP_720s(year):
    
    dataFrame = pd.DataFrame()
    currDate = datetime(year, 1, 11)
    pastDate = currDate + timedelta(days = -10)

    while currDate.year < year+1:
        
        pastDateStr = pastDate.strftime('%Y.%m.%d_%H:%M:%S')
        currDateStr = currDate.strftime('%Y.%m.%d_%H:%M:%S')
        
        url = pastDateStr + '-' + currDateStr
        print(url)
        
        client = drms.Client()
        dataFrameQuery = client.query(
        'hmi.sharp_720s[][' + url + ']',
        key='T_REC,HARPNUM,TOTUSJH,TOTPOT,TOTUSJZ,ABSNJZH,SAVNCPP,USFLUX,AREA_ACR,MEANPOT,R_VALUE,SHRGT45,MEANSHR,MEANGAM,NOAA_AR,QUALITY'
        )
        
        dataFrame = pd.concat([dataFrame, dataFrameQuery], ignore_index= True)
        currDate = currDate + timedelta(days = 10)
        pastDate = pastDate + timedelta(days = 10)
        
        if pastDate.year != currDate.year:
            currDate = datetime(pastDate.year, 12, 31, 23, 59, 59)
            
    dataFrame.to_csv('data/HMI_SHARP_720s/HMI_SHARP_720s_' + str(year) + '.csv')
    


,T_REC,HARPNUM,TOTUSJH,TOTPOT,TOTUSJZ,ABSNJZH,SAVNCPP,USFLUX,AREA_ACR,MEANPOT,...,MEANGBT,MEANGBZ,MEANGBH,MEANJZH,MEANJZD,MEANALP,NOAA_AR,NOAA_NUM,NOAA_ARS,QUALITY
0,2010.05.01_00:00:00_TAI,1,270.853,2.905448e+22,5.777592e+12,30.270,2.383632e+12,6.510776e+21,107.599098,2242.021,...,66.808,84.497,32.193,0.003103,-0.131873,0.009330,11067,1,11067,66560
1,2010.05.01_00:12:00_TAI,1,264.299,3.306497e+22,5.654726e+12,12.939,6.364629e+11,6.521054e+21,105.709244,2461.199,...,68.349,90.781,32.345,-0.001279,-0.113589,-0.004021,11067,1,11067,0
2,2010.05.01_00:24:00_TAI,1,307.572,3.353350e+22,6.488687e+12,11.460,1.174558e+12,6.917875e+21,111.348442,2365.145,...,67.682,89.127,32.411,0.001073,0.061197,0.003400,11067,1,11067,0
3,2010.05.01_00:36:00_TAI,1,297.073,3.071844e+22,6.193157e+12,17.791,2.019295e+12,6.973706e+21,108.848373,2194.344,...,67.166,85.321,31.966,0.001688,0.053302,0.005150,11067,1,11067,0
4,2010.05.01_00:48:00_TAI,1,301.562,2.774973e+22,5.797055e+12,3.262,8.275204e+11,7.228647e+21,104.785210,1970.685,...,64.805,76.349,32.647,0.000308,0.011571,0.000902,11067,1,11067,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6454,2010.05.14_23:12:00_TAI,20,16.936,1.011828e+21,4.120292e+11,1.278,5.367713e+10,2.740946e+20,16.831247,1621.050,...,130.600,131.578,42.694,-0.002720,0.212523,-0.012113,0,0,MISSING,0
6455,2010.05.14_23:24:00_TAI,20,16.079,9.428788e+20,3.739918e+11,1.471,6.560597e+10,2.602149e+20,16.506516,1639.666,...,129.289,130.678,42.932,-0.003396,0.281941,-0.014333,0,0,MISSING,0
6456,2010.05.14_23:36:00_TAI,20,15.391,8.729665e+20,3.658875e+11,1.239,6.533551e+10,2.490319e+20,16.632624,1576.336,...,128.077,129.905,44.698,-0.002971,0.291544,-0.012733,0,0,MISSING,0
6457,2010.05.14_23:48:00_TAI,20,15.232,9.196592e+20,3.752918e+11,1.021,1.777882e+10,2.490580e+20,15.650254,1660.651,...,127.111,128.139,45.210,-0.002448,0.079332,-0.010304,0,0,MISSING,0


In [9]:
#date format 2022.11.01_00:00:00-2022.11.02_00:00:00
def extractCgemLorentz(year):
    
    dataFrame = pd.DataFrame()
    currDate = datetime(year, 1, 11)
    pastDate = currDate + timedelta(days = -10)

    while currDate.year < year+1:
        
        pastDateStr = pastDate.strftime('%Y.%m.%d_%H:%M:%S')
        currDateStr = currDate.strftime('%Y.%m.%d_%H:%M:%S')
        
        url = pastDateStr + '-' + currDateStr
        #print(url)
        
        client = drms.Client()
        dataFrameQuery = client.query(
            'cgem.lorentz[][' + url + ']',
            key='T_REC,HARPNUM, TOTBSQ, TOTFZ, EPSZ, NOAA_AR, QUALITY'
        )
        
        dataFrame = pd.concat([dataFrame, dataFrameQuery], ignore_index = True)
        currDate = currDate + timedelta(days = 10)
        pastDate = pastDate + timedelta(days = 10)
        
        if pastDate.year != currDate.year:
            currDate = datetime(pastDate.year, 12, 31, 23, 59, 59)
            
    dataFrame.to_csv('data/cgemLorentz/cgemLorentz_' + str(year) + '.csv')
    


In [10]:
for i in range (2010, 2022+1):
    print ('Currently extracting data from the year ' + str(i))
    extractCgemLorentz(i)

Currently extracting data from the year 2010
Currently extracting data from the year 2011
Currently extracting data from the year 2012
Currently extracting data from the year 2013
Currently extracting data from the year 2014
Currently extracting data from the year 2015
Currently extracting data from the year 2016
Currently extracting data from the year 2017
Currently extracting data from the year 2018
Currently extracting data from the year 2019
Currently extracting data from the year 2020
Currently extracting data from the year 2021
Currently extracting data from the year 2022
